##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Assinaturas no TensorFlow Lite

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/lite/guide/signatures"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a> </td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/lite/guide/signatures.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a> </td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/lite/guide/signatures.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a> </td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/lite/guide/signatures.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a> </td>
</table>

O TensorFlow Lite tem suporte à conversão das especificações de entrada/saída de modelos do TensorFlow para o TensorFlow Lite. As especificações de entrada/saída são chamadas de "assinaturas", que podem ser especificadas ao compilar um SavedModel ou ao criar funções concretas.

As assinaturas no TensorFlow Lite contam com os seguintes recursos:

- Especificam entradas e saídas do modelo convertido para TensorFlow Lite, respeitando as assinaturas do modelo do TensorFlow.
- Permitem que um único modelo do TensorFlow Lite tenha suporte a diversos pontos de entrada.

A assinatura é composta por três componentes:

- Inputs (entradas): faz o mapeamento de entradas do nome de entradas na assinatura para um tensor de entrada.
- Outputs (saídas): faz o mapeamento de saídas do nome de saída na assinatura para um tensor de saída.
- Signature Key (chave de assinatura): nome que identifica um ponto de entrada do grafo.


## Configuração

In [ ]:
import tensorflow as tf

## Modelo de exemplo

Vamos supor que tenhamos duas tarefas, como codificação e decodificação, em um modelo do TensorFlow:

In [ ]:
class Model(tf.Module):

  @tf.function(input_signature=[tf.TensorSpec(shape=[None], dtype=tf.float32)])
  def encode(self, x):
    result = tf.strings.as_string(x)
    return {
         "encoded_result": result
    }

  @tf.function(input_signature=[tf.TensorSpec(shape=[None], dtype=tf.string)])
  def decode(self, x):
    result = tf.strings.to_number(x)
    return {
         "decoded_result": result
    }

Pela perspectiva de assinatura, o modelo do TensorFlow acima pode ser resumido da seguinte forma:

- Assinatura

    - Chave: encode
    - Entradas: {"x"}
    - Saída: {"encoded_result"}

- Assinatura

    - Chave: decode
    - Entradas: {"x"}
    - Saída: {"decoded_result"}

## Conversão de modelos com assinaturas

As APIs de conversão do TensorFlow Lite levam as informações de assinatura acima ao modelo convertido para TensorFlow Lite.

Essa funcionalidade de conversão está disponível em todas as APIs de conversão a partir do TensorFlow versão 2.7.0. Confira os exemplos de uso abaixo.


### Usando um SavedModel

In [ ]:
model = Model()

# Save the model
SAVED_MODEL_PATH = 'content/saved_models/coding'

tf.saved_model.save(
    model, SAVED_MODEL_PATH,
    signatures={
      'encode': model.encode.get_concrete_function(),
      'decode': model.decode.get_concrete_function()
    })

# Convert the saved model using TFLiteConverter
converter = tf.lite.TFLiteConverter.from_saved_model(SAVED_MODEL_PATH)
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,  # enable TensorFlow Lite ops.
    tf.lite.OpsSet.SELECT_TF_OPS  # enable TensorFlow ops.
]
tflite_model = converter.convert()

# Print the signatures from the converted model
interpreter = tf.lite.Interpreter(model_content=tflite_model)
signatures = interpreter.get_signature_list()
print(signatures)

### Usando um modelo do Keras

In [ ]:
# Generate a Keras model.
keras_model = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(2, input_dim=4, activation='relu', name='x'),
        tf.keras.layers.Dense(1, activation='relu', name='output'),
    ]
)

# Convert the keras model using TFLiteConverter.
# Keras model converter API uses the default signature automatically.
converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)
tflite_model = converter.convert()

# Print the signatures from the converted model
interpreter = tf.lite.Interpreter(model_content=tflite_model)

signatures = interpreter.get_signature_list()
print(signatures)

### De funções concretas

In [ ]:
model = Model()

# Convert the concrete functions using TFLiteConverter
converter = tf.lite.TFLiteConverter.from_concrete_functions(
    [model.encode.get_concrete_function(),
     model.decode.get_concrete_function()], model)
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,  # enable TensorFlow Lite ops.
    tf.lite.OpsSet.SELECT_TF_OPS  # enable TensorFlow ops.
]
tflite_model = converter.convert()

# Print the signatures from the converted model
interpreter = tf.lite.Interpreter(model_content=tflite_model)
signatures = interpreter.get_signature_list()
print(signatures)

## Execução de assinaturas

As APIs de inferência do TensorFlow têm suporte a execuções baseadas em assinatura:

- Acesso aos tensores de entrada/saída por meio dos nomes das entradas e saídas, especificadas pela assinatura.
- Execução de cada ponto de entrada do grafo separadamente, identificado pela chave de assinatura.
- Suporte ao procedimento de inicialização do SavedModel.

As vinculações das linguagens Java, C++ e Python já estão disponíveis. Confira os exemplos nas seções abaixo.


### Java

```
try (Interpreter interpreter = new Interpreter(file_of_tensorflowlite_model)) {
  // Run encoding signature.
  Map&lt;String, Object&gt; inputs = new HashMap&lt;&gt;();
  inputs.put("x", input);
  Map&lt;String, Object&gt; outputs = new HashMap&lt;&gt;();
  outputs.put("encoded_result", encoded_result);
  interpreter.runSignature(inputs, outputs, "encode");

  // Run decoding signature.
  Map&lt;String, Object&gt; inputs = new HashMap&lt;&gt;();
  inputs.put("x", encoded_result);
  Map&lt;String, Object&gt; outputs = new HashMap&lt;&gt;();
  outputs.put("decoded_result", decoded_result);
  interpreter.runSignature(inputs, outputs, "decode");
}
```

### C++

```
SignatureRunner* encode_runner =
    interpreter->GetSignatureRunner("encode");
encode_runner->ResizeInputTensor("x", {100});
encode_runner->AllocateTensors();

TfLiteTensor* input_tensor = encode_runner->input_tensor("x");
float* input = GetTensorData<float>(input_tensor);
// Fill `input`.

encode_runner->Invoke();

const TfLiteTensor* output_tensor = encode_runner->output_tensor(
    "encoded_result");
float* output = GetTensorData<float>(output_tensor);
// Access `output`.
```

### Python

In [ ]:
# Load the TFLite model in TFLite Interpreter
interpreter = tf.lite.Interpreter(model_content=tflite_model)

# Print the signatures from the converted model
signatures = interpreter.get_signature_list()
print('Signature:', signatures)

# encode and decode are callable with input as arguments.
encode = interpreter.get_signature_runner('encode')
decode = interpreter.get_signature_runner('decode')

# 'encoded' and 'decoded' are dictionaries with all outputs from the inference.
input = tf.constant([1, 2, 3], dtype=tf.float32)
print('Input:', input)
encoded = encode(x=input)
print('Encoded result:', encoded)
decoded = decode(x=encoded['encoded_result'])
print('Decoded result:', decoded)

## Limitações conhecidas

- Como o interpretador do TF Lite não garante a segurança de threads, os executadores de assinaturas do mesmo interpretador não serão executados simultaneamente.
- Ainda não há suporte ao C/iOS/Swift.


## Atualizações

- Versão 2.7
    - Recurso de múltiplas assinaturas implementado.
    - Todas as APIs de conversão a partir da versão 2 geram modelos do TensorFlow Lite com assinaturas.
- Versão 2.5
    - O recurso de assinaturas está disponível por meio da API de conversão `from_saved_model`.